In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c estnltk estnltk

# Tekstide metaandmete allalaadimine andmebaasist

Valitud korpuse tekstid salvestatakse TSV-faili 

**Tulbad**:

1. collectionId
2. metaandmed file
3. metaandmed type
4. metaandmed subcorpus
5. metaandmed kogu meta json formaadis

Näide:





## Vajalikud teegid

In [2]:
from __future__ import print_function
#from estnltk import Text, logger
from estnltk.storage.postgres import PostgresStorage
import sqlite3
import sys

## Abifunktsioonid

In [3]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)


Salvestab kollektsioonide "järelhoidja" baasi.

In [4]:
def saveToDisk(rows, lastcollection, fOut):
   
    global TABLENAME, cursor, connection

    fOut.write('\n'.join( '\t'.join(s) for s in rows) + '\n')

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection, ) )
    
    connection.commit()
    eprint(f'TSV-sse salvestatud kollokatsioonid: 0 - {lastcollection}' )

In [5]:
def prepareDb():
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq
    ON collections_processed(tablename);
    """)


In [6]:
def getPlaceholder(tablename):
    cursor.execute(f"""SELECT lastcollection FROM collections_processed WHERE tablename = ?""", (tablename,) );
    lastcollection = cursor.fetchone()

    fileMode = 'w'
    if not lastcollection:
        lastcollection = -1
    else:
        #andmete kogumine jäi pooleli ning jätkatakse samasse faili kirjutamist
        lastcollection = lastcollection[0]
        fileMode = 'a'
    return (lastcollection, fileMode)


## Muutujad

In [7]:
collectionName = 'koondkorpus_base_subset_of_5000_v2' 
#collectionName = 'koondkorpus_base_v2' 


Fail, kuhu väljund salvestatakse.

In [8]:
outputfile = f'metadata_{collectionName}.tsv'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus faili.

In [9]:
BATCHSIZE = 1000

In [10]:
TABLENAME = f'metadata_{collectionName}'

## Andmebaasi loomine ja ette valmistamine

In [11]:
connection = sqlite3.connect("collect_texts.db") #
cursor = connection.cursor()

prepareDb()

## Tekstide alla laadimine

In [12]:
storage = PostgresStorage(pgpass_file='~/.pgpass',
                          schema='estonian_text_corpora',
                          role='estonian_text_corpora_read',
                          temporary=False)

INFO:storage.py:42: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [13]:
collection = storage[collectionName]
TOTALROWS = len(collection)

In [14]:
print (collection)

Kontrollitakse, mitmenda kollektsiooni juurde skript eelmine kord jäi, juhul kui skripti töö katkes ootamatult. Nullist alustamiseks tuleb kustutada skripti loodud *collect_texts.db* fail.

In [15]:
import json
(lastcollection, fileMode) = getPlaceholder(TABLENAME)

#mitu kollektsiooni jäi alla tõmbamata
unprocessed = TOTALROWS - lastcollection - 1

print (f'{unprocessed} out of {TOTALROWS} collections to download')
fOut = open(outputfile, fileMode)

###
iterations = 0
unsaved = 0
rows = []
#keys = []
keys = ['file', 'type', 'subcorpus']
if lastcollection == 0:
    rows.append ( ['colId', 'file', 'type', 'subcorpus', 'meta_json'])
for (colId, text) in collection.select (progressbar='notebook', layers=[], return_index=True ).tail(unprocessed):
 
    
    iterations +=1
    row = [str(colId)]
    for key in keys:
        row.append(str(text.meta[key]))
    unsaved =   1
    row.append(json.dumps(text.meta))  
    rows.append(row)
    if not iterations%BATCHSIZE:
        saveToDisk(rows, colId, fOut)
        rows = []
        unsaved = 0

if unsaved: saveToDisk(rows, colId, fOut)
fOut.close()

5000 out of 5000 collections to download


TSV-sse salvestatud kollokatsioonid: 0 - 999
TSV-sse salvestatud kollokatsioonid: 0 - 1999
TSV-sse salvestatud kollokatsioonid: 0 - 2999
TSV-sse salvestatud kollokatsioonid: 0 - 3999


TSV-sse salvestatud kollokatsioonid: 0 - 4999


In [16]:
import subprocess
batcmd=f"wc {outputfile}"
res = subprocess.check_output(batcmd, shell=True)

In [17]:
batcmd


'wc metadata_koondkorpus_base_subset_of_5000_v2.tsv'

In [18]:
res

b'    5000  128369 1438488 metadata_koondkorpus_base_subset_of_5000_v2.tsv\n'